Since our research falls within the category of quantitative studies, we have adhered to the MISQ's guidelines for this category to organize the transparency materials.

# Data collection procedures and materials
In this study, the quantitative data, including secondary sources such as online review content and reviewer information, were obtained directly from the enterprise (dianping.com) through a cooperative agreement. The data provided by the enterprise were already organized and presented in a structured format. Consequently, no further processing was required for the purposes of this research. Therefore, additional details regarding data collection procedures and materials are not necessary in this article.

Although it is not necessary for us to provide detailed data collection procedures and materials, we can offer metadata about the dataset to assist readers in understanding and referencing the data. This metadata includes information such as the timeframe of data collection, the types of data collected (e.g., review content, reviewer profiles), the structure and format of the dataset.

By providing this metadata, we aim to enhance the transparency and reproducibility of the study, enabling readers to better comprehend the scope and characteristics of the data used, even in the absence of direct access to the dataset itself.

# Data
The data used in this study cannot be provided due to the following reasons:

1. Sensitivity of Individual Data: The dataset contains highly sensitive information related to individual users, which raises significant privacy concerns.
2. Non-Disclosure Agreement (NDA): Access to the data was granted under the terms of a strict non-disclosure agreement with the enterprise, which prohibits the sharing or redistribution of the dataset.

As an alternative, we will outline how readers can obtain the same or similar data. Specifically, readers can access comparable datasets by visiting the enterprise’s official website (dianping.com). The platform provides publicly available information, such as online review content and reviewer profiles, which may serve as a valuable substitute for the data utilized in this study. 

To access the same or similar data, readers or users must register and become members of the enterprise’s platform, dianping.com. As registered members, they can browse the data available on the platform at the current point in time. However, the right to browse the data is not equivalent to the right to collect and use the data. The readers should respect all rules or legal obligations offered by the platform if they want to collect its data.

It should be noted, however, that some data included in this study’s dataset (from 2015 to 2017) may no longer be available on the platform due to deletions by the enterprise or individual users. While these deleted records are no longer accessible, they were relevant and valuable to the research conducted in this study and were therefore included in the analysis.

Despite these limitations, the proposed method provides sufficient access to comparable data for readers seeking to explore similar research questions or verify the findings of this study.

# Data analytic methods
In this study, we employ a nonlinear multilevel hierarchical model for our empirical research. The primary methodological references for this approach are drawn from articles published in the R Journal. While we are unable to disclose the actual dataset used due to confidentiality constraints, we ensure the transparency of our research by making the model and corresponding (pseudo)code publicly available. This approach not only facilitates reproducibility but also provides a framework for researchers to adapt and apply similar methodologies to their respective datasets.

The use of a nonlinear multilevel hierarchical model is particularly suitable for capturing the complex, nested structure of our data, as well as for accommodating potential nonlinear relationships. By referencing state-of-the-art techniques documented in the R Journal (all necessary references have been included in the manuscript), we align our work with established best practices in statistical modeling and computation. The pseudocode provided aims to offer clear and detailed guidance on the implementation of the model, thereby enabling other researchers to replicate our analytical framework or extend it to new contexts.

# Quantitative primary data
First, we present the processes and code used for reading experimental data, conducting balance checks, and calculating reliability and validity.

In [ ]:
from factor_analyzer import (
    ConfirmatoryFactorAnalyzer, ModelSpecificationParser)
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.stats import chi2_contingency
from statsmodels.stats.mediation import Mediation
import statsmodels.api as sm
from factor_analyzer import FactorAnalyzer
from scipy import stats
from tqdm import tqdm
import pandas as pd
import numpy as np
np.random.seed(158)
pd.set_option('display.max_columns', 40)
tqdm.pandas(desc='pandas bar')
# usage: df.progress_apply()

# %% load data
df = pd.read_csv('exdata.csv', header=0, encoding='gbk')
df = df.dropna()
print(df.head())
print(df.describe())
print(df.columns)
'''
Index(['group', 'gender', 'age', 'job', 'city', 'education', 'salary',
       'satiety1', 'tval', 'taro', 'pval', 'paro', 'pf1-', 'pf2', 'pf3',
       'pf4-', 'pf5', 'ce1', 'ce2', 'ce3', 'a1', 'a2', 'a3', 'a4', 'pa', 'wr1',    
       'wr2', 'wr3', 'app', 'avo', 'phea', 'ppal', 'ph1', 'ph2', 'ph3', 'pi1',     
       'pi2', 'pi3', 'tval_sr', 'taro_sr', 'pval_sr', 'paro_sr', 'pf1-_sr',        
       'pf2_sr', 'pf3_sr', 'pf4-_sr', 'pf5_sr', 'ce1_sr', 'ce2_sr', 'ce3_sr',      
       'a1_sr', 'a2_sr', 'a3_sr', 'a4_sr', 'pa_sr', 'wr1_sr', 'wr2_sr',
       'wr3_sr', 'app_sr', 'avo_sr', 'phea_sr', 'ppal_sr', 'ph1_sr', 'ph2_sr',     
       'ph3_sr', 'pi1_sr', 'pi2_sr', 'pi3_sr', 'satiety2', 'id_p', 'id_i'],        
      dtype='object')
'''

df1 = pd.read_csv('data1.csv', header=0)
print(df1.columns)
df1['PicAro2'] = df1['PicAro'] * df1['PicAro']
df1['TextAro2'] = df1['TextAro'] * df1['TextAro']
mod = sm.OLS.from_formula(
    "Helpfulness ~ 1 + PicVal + PicAro + PicAro2 + TextVal + TextAro + TextAro2 + C(SP) + C(AT) + C(SU)", df1)
print(mod.fit().summary())


# %% basic information
print(df.groupby('gender').size())
'''
gender
女    230
男    192
'''
print(df.groupby('age').size())
'''
age
20岁及以下     83
21-39岁    289
40岁及以上     50
'''
print(df.groupby('salary').size())
'''
salary
20万及以上    101
5万-20万    108
5万及以下     213
'''
print(df.groupby('education').size())
'''
education
专科或职业教育     38
本科         215
研究生及以上     151
高中及以下       18
'''

# %% balance check
df.groupby('group')['gender'].value_counts()
df.groupby('group')['age'].value_counts()
df.groupby('group')['salary'].value_counts()
df.groupby('group')['education'].value_counts()
cross_tab = {attri: pd.crosstab(df['group'], df[attri])
             for attri in ['gender', 'age', 'salary', 'education']}
balance_tab = pd.concat(cross_tab.values(), axis=1)

chisq_res = {attri: chi2_contingency(cross_tab[attri])
             for attri in ['gender', 'age', 'salary', 'education']}
# print(chisq_res)
'''
'gender': (18.16703216998806, 0.2539860477044757, 15)
'age': (30.217701330671503, 0.45454521350793303, 30)
'salary': (36.24127531879915, 0.20028380890079578, 30)
'education': (43.03444512476223, 0.5555432275471534, 45)
'''

# %% Cronbach's alphas (reliability)


def C_alpha(var_frame):
    cors = var_frame.corr()
    # number of questions
    N = cors.shape[0]
    # 2.2 Calculate R
    # For this, we'll loop through the columns and append every
    # relevant correlation to an array calles "r_s". Then, we'll
    # calculate the mean of "r_s"
    rs = np.array([])
    for i, col in enumerate(cors.columns):
        sum_ = cors[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)
   # 3. Use the formula to calculate Cronbach's Alpha
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    return cronbach_alpha


# for the 1st review
df['pf1'] = 7 - df['pf1-']
df['pf4'] = 7 - df['pf4-']
# print(C_alpha(df[['pf1','pf2','pf3','pf4','pf5']]))
# print(C_alpha(df[['pf2','pf3','pf5']]))
# print(C_alpha(df[['ce1','ce2','ce3']]))
# print(C_alpha(df[['a1','a2','a3','a4']]))
# print(C_alpha(df[['wr1','wr2','wr3']]))
# print(C_alpha(df[['ph1','ph2','ph3']]))
# print(C_alpha(df[['pi1','pi2','pi3']]))
'''
0.8591707541611933
0.8860333878166867
0.8807747263608968
0.8828780218307613
0.8892377878815275
0.9197336378508992
0.9696365372971941
'''

# for the 2nd review
df['pf1_sr'] = 7 - df['pf1-_sr']
df['pf4_sr'] = 7 - df['pf4-_sr']
# print(C_alpha(df[['pf1_sr','pf2_sr','pf3_sr','pf4_sr','pf5_sr']]))
# print(C_alpha(df[['pf2_sr','pf3_sr','pf5_sr']]))
# print(C_alpha(df[['ce1_sr','ce2_sr','ce3_sr']]))
# print(C_alpha(df[['a1_sr','a2_sr','a3_sr','a4_sr']]))
# print(C_alpha(df[['wr1_sr','wr2_sr','wr3_sr']]))
# print(C_alpha(df[['ph1_sr','ph2_sr','ph3_sr']]))
# print(C_alpha(df[['pi1_sr','pi2_sr','pi3_sr']]))
'''
0.9090818090518659
0.9146989257990688
0.9133944720786907
0.8779809460605016
0.8944337215512985
0.936490351776063
0.979782955766911
'''

# %% manipulation check 1
pic_df = df[df['group'].apply(
    lambda x: x in [1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15])]
tex_df = df[df['group'].apply(lambda x: x in [4, 8, 12, 16])]

tv, pv = stats.ttest_ind(df[df['tvt1'] == 1]['tval'],
                         df[df['group'] >= 13]['tval'])
print(str(pv/2))  # one-tailed t-test
print(df[df['tvt1'] == 1]['tval'].mean())
print(df[df['group'] >= 13]['tval'].mean())

tv, pv = stats.ttest_ind(df[df['tvt1'] == 1]['tval'],
                         df[df['tvt1'] == 0]['taro'])
print("wtf" + str(pv/2))  # one-tailed t-test


tv, pv = stats.ttest_ind(df[df['tvt1'] == 1]['taro'],
                         df[df['tvt1'] == 0]['taro'])
print(str(pv/2))  # one-tailed t-test
print(df[df['tvt1'] == 1]['taro'].mean())
print(df[df['tvt1'] == 0]['taro'].mean())

tv, pv = stats.ttest_ind(pic_df[pic_df['pt1'] == 'G']['pval'],
                         pic_df[pic_df['pt1'] == 'R']['pval'])
print(str(pv/2))  # one-tailed t-test
print(pic_df[pic_df['pt1'] == 'G']['pval'].mean())
print(pic_df[pic_df['pt1'] == 'R']['pval'].mean())

tv, pv = stats.ttest_ind(pic_df[pic_df['pt1'] == 'G']['paro'],
                         pic_df[pic_df['pt1'] == 'R']['paro'],)
print(str(pv/2))  # one-tailed t-test
print(pic_df[pic_df['pt1'] == 'G']['paro'].mean())
print(pic_df[pic_df['pt1'] == 'R']['paro'].mean())

tv, pv = stats.ttest_ind(pic_df[pic_df['pt1'] == 'G']['pval'],
                         pic_df[pic_df['pt1'] == 'G']['paro'],)
print("wtf" + str(pv/2))  # one-tailed t-test

# %% manipulation check 2
tv, pv = stats.ttest_ind(pic_df[pic_df['pt1'] == 'G']['ppal'],
                         pic_df[pic_df['pt1'] == 'O']['ppal'])
print(str(pv/2))  # one-tailed t-test
print(pic_df[pic_df['pt1'] == 'G']['ppal'].mean())
print(pic_df[pic_df['pt1'] == 'O']['ppal'].mean())

tv, pv = stats.ttest_ind(pic_df[pic_df['pt1'] == 'G']['phea'],
                         pic_df[pic_df['pt1'] == 'O']['phea'])
print(str(pv/2))  # one-tailed t-test
print(pic_df[pic_df['pt1'] == 'G']['phea'].mean())
print(pic_df[pic_df['pt1'] == 'O']['phea'].mean())

tv, pv = stats.ttest_ind(pic_df['taro'],
                         pic_df['paro'])
print(str(pv/2))  # one-tailed t-test
print(pic_df['taro'].mean())
print(pic_df['paro'].mean())


# %% Confirmatory factor analysis (validity) without pf1, pf4
temp = df[['pf2', 'pf3',  'pf5', 'ce1', 'ce2', 'ce3', 'a1', 'a2',
           'a3', 'a4', 'wr1', 'wr2', 'wr3', 'ph1', 'ph2', 'ph3', 'pi1', 'pi2', 'pi3']]
model_dict = {"F1": ['pf2', 'pf3', 'pf5'],
              "F2": ['ce1', 'ce2', 'ce3'],
              "F3": ['a1', 'a2', 'a3', 'a4'],
              "F4": ['wr1', 'wr2', 'wr3'],
              "F5": ['ph1', 'ph2', 'ph3'],
              "F6": ['pi1', 'pi2', 'pi3']}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
# print(loadings_se)
'''
[[0.05088133 0.         0.         0.         0.         0.        ]
 [0.05015584 0.         0.         0.         0.         0.        ]
 [0.05904468 0.         0.         0.         0.         0.        ]
 [0.         0.05111221 0.         0.         0.         0.        ]
 [0.         0.05234574 0.         0.         0.         0.        ]
 [0.         0.05075873 0.         0.         0.         0.        ]
 [0.         0.         0.05154054 0.         0.         0.        ]
 [0.         0.         0.04858426 0.         0.         0.        ]
 [0.         0.         0.05545921 0.         0.         0.        ]
 [0.         0.         0.05814206 0.         0.         0.        ]
 [0.         0.         0.         0.05917729 0.         0.        ]
 [0.         0.         0.         0.05045251 0.         0.        ]
 [0.         0.         0.         0.05449334 0.         0.        ]
 [0.         0.         0.         0.         0.04671028 0.        ]
 [0.         0.         0.         0.         0.04760733 0.        ]
 [0.         0.         0.         0.         0.04712727 0.        ]
 [0.         0.         0.         0.         0.         0.04421903]
 [0.         0.         0.         0.         0.         0.04683588]
 [0.         0.         0.         0.         0.         0.04727852]]
'''
error_vars_se = np.sqrt(cfa.error_vars_.reshape(19,))
# print(cfa.factor_varcovs_)
'''
[[1.         0.0675506  0.1388678  0.09007235 0.09670755 0.03067129]
 [0.0675506  1.         0.29058941 0.26444203 0.29339554 0.1406637 ]
 [0.1388678  0.29058941 1.         0.37179055 0.35404266 0.1626066 ]
 [0.09007235 0.26444203 0.37179055 1.         0.34159494 0.16602322]
 [0.09670755 0.29339554 0.35404266 0.34159494 1.         0.08908992]
 [0.03067129 0.1406637  0.1626066  0.16602322 0.08908992 1.        ]]
 '''

loads = loadings.reshape(114,)[[0, 6, 12, 19, 25, 31, 38, 44,
                                50, 56, 63, 69, 75, 82, 88, 94, 101, 107, 113]]
# print(sum(loads[:3]**2)/(sum(loads[:3]**2)+ sum(error_vars_se[:3]**2)))
# print(sum(loads[3:6]**2)/(sum(loads[3:6]**2)+ sum(error_vars_se[3:6]**2)))
# print(sum(loads[6:10]**2)/(sum(loads[6:10]**2)+ sum(error_vars_se[6:10]**2)))
# print(sum(loads[10:13]**2)/(sum(loads[10:13]**2)+ sum(error_vars_se[10:13]**2)))
# print(sum(loads[13:16]**2)/(sum(loads[13:16]**2)+ sum(error_vars_se[13:16]**2)))
# print(sum(loads[16:19]**2)/(sum(loads[16:19]**2)+ sum(error_vars_se[16:19]**2)))

'''
0.6804573617420825
0.6848158607818193
0.633622713634425
0.6700579054797045
0.7420011115386957
0.81761304370953
'''

# %% Confirmatory factor analysis (validity) with pf1, pf4
temp = df[['pf1-', 'pf2', 'pf3', 'pf4-', 'pf5', 'ce1', 'ce2', 'ce3', 'a1', 'a2',
           'a3', 'a4', 'wr1', 'wr2', 'wr3', 'ph1', 'ph2', 'ph3', 'pi1', 'pi2', 'pi3']]
model_dict = {"F1": ['pf1-', 'pf2', 'pf3', 'pf4-', 'pf5'],
              "F2": ['ce1', 'ce2', 'ce3'],
              "F3": ['a1', 'a2', 'a3', 'a4'],
              "F4": ['wr1', 'wr2', 'wr3'],
              "F5": ['ph1', 'ph2', 'ph3'],
              "F6": ['pi1', 'pi2', 'pi3']}
model_spec = ModelSpecificationParser.parse_model_specification_from_dict(
    temp, model_dict)
cfa = ConfirmatoryFactorAnalyzer(model_spec, disp=False)
cfa.fit(temp.values)
loadings = cfa.loadings_
loadings_se, error_vars_se = cfa.get_standard_errors()
# print(loadings_se)
'''
[[0.07149457 0.         0.         0.         0.         0.        ]
 [0.05220995 0.         0.         0.         0.         0.        ]
 [0.05272207 0.         0.         0.         0.         0.        ]
 [0.06326951 0.         0.         0.         0.         0.        ]
 [0.05843026 0.         0.         0.         0.         0.        ]
 [0.         0.05086663 0.         0.         0.         0.        ]
 [0.         0.05212719 0.         0.         0.         0.        ]
 [0.         0.05082515 0.         0.         0.         0.        ]
 [0.         0.         0.05106664 0.         0.         0.        ]
 [0.         0.         0.04817152 0.         0.         0.        ]
 [0.         0.         0.05431391 0.         0.         0.        ]
 [0.         0.         0.05646783 0.         0.         0.        ]
 [0.         0.         0.         0.05760128 0.         0.        ]
 [0.         0.         0.         0.05025958 0.         0.        ]
 [0.         0.         0.         0.05351047 0.         0.        ]
 [0.         0.         0.         0.         0.04550612 0.        ]
 [0.         0.         0.         0.         0.04579569 0.        ]
 [0.         0.         0.         0.         0.04709788 0.        ]
 [0.         0.         0.         0.         0.         0.04314606]
 [0.         0.         0.         0.         0.         0.04494906]
 [0.         0.         0.         0.         0.         0.04553063]]
'''
# print(error_vars_se)
'''
[0.10417787 0.05080782 0.05413315 0.0787603  0.06493564 0.0486113
 0.05104546 0.04852934 0.04751227 0.04037249 0.05346928 0.06154693
 0.06366381 0.04737554 0.05350057 0.03375969 0.0341493  0.03835605
 0.02154999 0.02330404 0.0242826 ]
 '''

error_vars_se = np.sqrt(cfa.error_vars_.reshape(21,))
# print(cfa.factor_varcovs_)
'''
[[ 1.          0.03718711  0.09997903  0.04022141  0.07020146 -0.00843205]
 [ 0.03718711  1.          0.28030712  0.24365414  0.29265758  0.13512102]
 [ 0.09997903  0.28030712  1.          0.36289666  0.35920525  0.15644118]
 [ 0.04022141  0.24365414  0.36289666  1.          0.33421972  0.15825606]
 [ 0.07020146  0.29265758  0.35920525  0.33421972  1.          0.07365902]
 [-0.00843205  0.13512102  0.15644118  0.15825606  0.07365902  1.        ]]
 '''

loads = loadings.reshape(126,)[[0, 6, 12, 18, 24, 31, 37, 43,
                                50, 56, 62, 68, 75, 81, 87, 94, 100, 106, 113, 119, 125]]
# 126 = 21 * 6, corresponds to the non-zero term of loadings_se

# print(sum(loads[:5]**2)/(sum(loads[:5]**2)+ sum(error_vars_se[:5]**2)))
# print(sum(loads[5:8]**2)/(sum(loads[5:8]**2)+ sum(error_vars_se[5:8]**2)))
# print(sum(loads[8:12]**2)/(sum(loads[8:12]**2)+ sum(error_vars_se[8:12]**2)))
# print(sum(loads[12:15]**2)/(sum(loads[12:15]**2)+ sum(error_vars_se[12:15]**2)))
# print(sum(loads[15:18]**2)/(sum(loads[15:18]**2)+ sum(error_vars_se[15:18]**2)))
# print(sum(loads[18:21]**2)/(sum(loads[18:21]**2)+ sum(error_vars_se[18:21]**2)))

'''
0.5585855030001372
0.6827916679027678
0.639818985285587
0.6774083254145898
0.7506165612998753
0.8491876427762699
'''

# values above 0.7 are considered very good
# the AVE of each of the latent constructs should be higher than the highest squared correlation with any other latent variable.
# If that is the case, discriminant validity is established on the construct level.
# In our case, the highest squared correlation is 0.35

# %% construct mean variables
df['pf'] = (df['pf1-'] + df['pf2'] + df['pf3'] + df['pf4-'] + df['pf5']) / 5
df['pfn'] = (df['pf2'] + df['pf3'] + df['pf5']) / 3
df['ce'] = (df['ce1'] + df['ce2'] + df['ce3']) / 3
df['a'] = (df['a1'] + df['a2'] + df['a3'] + df['a4']) / 4
df['wr'] = (df['wr1'] + df['wr2'] + df['wr3']) / 3
df['ph'] = (df['ph1'] + df['ph2'] + df['ph3']) / 3
df['pi'] = (df['pi1'] + df['pi2'] + df['pi3']) / 3
df['pic_a'] = 7 - df['pa']
df['paro2'] = df['paro'] * df['paro']
df['paro3'] = (df['paro']-4) * (df['paro']-4)
df['taro2'] = df['taro'] * df['taro']
df['pf1_sr'] = 7 - df['pf1-_sr']
df['pf4_sr'] = 7 - df['pf4-_sr']
df['pf_sr'] = (df['pf1_sr'] + df['pf2_sr'] + df['pf3_sr'] +
               df['pf4_sr'] + df['pf5_sr']) / 5
df['ce_sr'] = (df['ce1_sr'] + df['ce2_sr'] + df['ce3_sr']) / 3
df['a_sr'] = (df['a1_sr'] + df['a2_sr'] + df['a3_sr'] + df['a4_sr']) / 4
df['wr_sr'] = (df['wr1_sr'] + df['wr2_sr'] + df['wr3_sr']) / 3
df['ph_sr'] = (df['ph1_sr'] + df['ph2_sr'] + df['ph3_sr']) / 3
df['pi_sr'] = (df['pi1_sr'] + df['pi2_sr'] + df['pi3_sr']) / 3
df['pic_a_sr'] = 7 - df['pa_sr']
df['tpv'] = abs(df['tval'] - df['pval'])
df['tpa'] = abs(df['taro'] - df['paro'])
df['vx'] = df['tval'] * df['pval']
df['ax'] = df['taro'] * df['paro']
df['Ipa'] = df['pval'] * (df['pval'] < 3).astype(int)
df['Ita'] = df['tval'] * (df['tval'] < 3).astype(int)

pic_df = df[df['group'].apply(
    lambda x: x in [1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15])]
tex_df = df[df['group'].apply(lambda x: x in [4, 8, 12, 16])]
pic_df_nonr = df[df['group'].apply(
    lambda x: x in [2, 3, 6, 7, 10, 11, 14, 15])]
regs = {}  # a dict to store regression results

Next, we present the processes and code used for reading text pretest data and conducting a manipulation check. The primary goal of this process is to demonstrate that our manipulation of the text achieved the intended effect.

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
data = pd.read_csv("Sum.csv")
data.describe()
data.columns

Index(['T', 'UserID', 'UserNo', 'IP Address', 'Started', 'Ended', 'Pid',
       'EV1_1', 'EV2_1', 'EV3_1', 'F1', 'F2', 'F3', 'EV1_2', 'EV2_2', 'EV3_2',
       'EV1_3', 'EV2_3', 'EV3_3', 'Gender', 'Age', 'Education', 'Income',
       'Race', 'Ideology', 'Satiety'],
      dtype='object')

In [ ]:
L_1 = 1/3 * (data[data["T"] == "L"]["EV1_1"] + data[data["T"] == "L"]["EV2_1"] + data[data["T"] == "L"]["EV3_1"])
L_2 = 1/3 * (data[data["T"] == "L"]["EV1_2"] + data[data["T"] == "L"]["EV2_2"] + data[data["T"] == "L"]["EV3_2"])
L_3 = 1/3 * (data[data["T"] == "L"]["EV1_3"] + data[data["T"] == "L"]["EV2_3"] + data[data["T"] == "L"]["EV3_3"])

B_1 = 1/3 * (data[data["T"] == "B"]["EV1_1"] + data[data["T"] == "B"]["EV2_1"] + data[data["T"] == "B"]["EV3_1"])
B_2 = 1/3 * (data[data["T"] == "B"]["EV1_2"] + data[data["T"] == "B"]["EV2_2"] + data[data["T"] == "B"]["EV3_2"])
B_3 = 1/3 * (data[data["T"] == "B"]["EV1_3"] + data[data["T"] == "B"]["EV2_3"] + data[data["T"] == "B"]["EV3_3"])

H_1 = 1/3 * (data[data["T"] == "H"]["EV1_1"] + data[data["T"] == "H"]["EV2_1"] + data[data["T"] == "H"]["EV3_1"])
H_2 = 1/3 * (data[data["T"] == "H"]["EV1_2"] + data[data["T"] == "H"]["EV2_2"] + data[data["T"] == "H"]["EV3_2"])
H_3 = 1/3 * (data[data["T"] == "H"]["EV1_3"] + data[data["T"] == "H"]["EV2_3"] + data[data["T"] == "H"]["EV3_3"])

VH_1 = 1/3 * (data[data["T"] == "VH"]["EV1_1"] + data[data["T"] == "VH"]["EV2_1"] + data[data["T"] == "VH"]["EV3_1"])
VH_2 = 1/3 * (data[data["T"] == "VH"]["EV1_2"] + data[data["T"] == "VH"]["EV2_2"] + data[data["T"] == "VH"]["EV3_2"])
VH_3 = 1/3 * (data[data["T"] == "VH"]["EV1_3"] + data[data["T"] == "VH"]["EV2_3"] + data[data["T"] == "VH"]["EV3_3"])

t_statistic, p_value = ttest_ind(H_1, VH_1)
print("%.2f" % L_1.mean())
print("%.2f" % B_1.mean())
print("%.2f" % H_1.mean())
print("%.2f" % VH_1.mean())
p_value = p_value / 2
print("%.2f" % p_value)

2.60
4.69
5.72
6.21
0.02

# Result
| Review No. | Treatment | Mean | P-value | N |
| :---: | :---: | :---: | :---: | :---: |
| 1 | VL  | 2.60 | 0.00 | 30 |
| 1 | L  | 4.69 | 0.00 | 25 |
| 1 | M  | 5.72 | 0.02 | 27 |
| 1 | H | 6.21 | - | 34 |
| 2 | VL  | 1.68 | 0.00 | 30 |
| 2 | L  | 3.44 | 0.00 | 25 |
| 2 | M  | 5.81 | 0.08 | 27 |
| 2 | H | 6.21 | - | 34 |
| 3 | VL  | 1.71 | 0.00 | 30 |
| 3 | L  | 4.15 | 0.00 | 25 |
| 3 | M  | 5.46 | 0.04 | 27 |
| 3 | H | 6.11 | - | 34 |

- VL = Very Low valence
- L = Low valence
- M = Median valence
- H = High valence

Next, we present the processes and code used for reading picture pretest data and conducting a manipulation check. The primary goal of this process is to demonstrate that our manipulation of the pictures achieved the intended effect.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

df_v = pd.read_csv('V_data.csv', header = 0)
df_a = pd.read_csv('A_data.csv', header = 0)
df_o = pd.read_csv('O_data.csv', header = 0)
df_v = df_v.dropna()
# print(df_v.head())
# print(df_v.describe())
print(df_v.columns)
print(df_a.columns)
print(df_o.columns)

Index(['Treatment', 'UserID', 'UserNo', 'IP Address', 'Started', 'Ended', 'CC',
       'Pid', 'P1v', 'F1v', 'P2v', 'F2v', 'P3v', 'F3v', 'P4v', 'AC1', 'P5v',
       'F4v', 'P6v', 'F5v', 'P7v', 'F6v', 'P8v', 'AC2', 'P9v', 'F7v', 'P10v',
       'MC', 'Gender', 'Age', 'Education', 'Income', 'Race', 'Ideology',
       'Satiety'],
      dtype='object')
Index(['Treatment', 'UserID', 'UserNo', 'IP Address', 'Started', 'Ended', 'CC',
       'Pid', 'P1a', 'F1a', 'P2a', 'F2a', 'P3a', 'F3a', 'P4a', 'AC1', 'P5a',
       'F4a', 'P6a', 'F5a', 'P7a', 'F6a', 'P8a', 'AC2', 'P9a', 'F7a', 'P10a',
       'MC', 'Gender', 'Age', 'Education', 'Income', 'Race', 'Ideology',
       'Satiety'],
      dtype='object')
Index(['Treatment', 'UserID', 'UserNo', 'IP Address', 'Started', 'Ended', 'CC',
       'Pid', 'P1Q', 'P1A1', 'P1A2', 'P1A3', 'P1B1', 'P1B2', 'P1B3', 'P1B4',
       'P1B5', 'P2Q', 'P2A1', 'P2A2', 'P2B1', 'P2B2', 'P2B3', 'P3Q', 'P3A1',
       'P3A2', 'P3A3', 'P3B1', 'P3B2', 'P3B3', 'P3B4', 'P3B5', 'P4Q', 'P4A1',
       'P4A2', 'P4B1', 'P4B2', 'P4B3', 'P5Q', 'P5A1', 'P5A2', 'P5A3', 'P5B1',
       'P5B2', 'P5B3', 'P5B4', 'P5B5', 'P6Q', 'P6A1', 'P6A2', 'P6B1', 'P6B2',
       'P6B3', 'P7Q', 'P7A1', 'P7A2', 'P7A3', 'P7B1', 'P7B2', 'P7B3', 'P7B4',
       'P7B5', 'P8Q', 'P8A1', 'P8A2', 'P8B1', 'P8B2', 'P8B3', 'P9Q', 'P9A1',
       'P9A2', 'P9A3', 'P9B1', 'P9B2', 'P9B3', 'P9B4', 'P9B5', 'P10Q', 'P10A1',
       'P10A2', 'P10B1', 'P10B2', 'P10B3', 'MC1', 'MC2', 'Gender', 'Age',
       'Education', 'Income', 'Race', 'Ideology', 'Satiety'],
      dtype='object')

In [ ]:
f_statistic, p_value = stats.f_oneway(
    df_v['P6v'][df_v['Treatment'] == 'cdp'],
    df_v['P6v'][df_v['Treatment'] == 'o']
)

print(f'F-statistic: {f_statistic}, p-value: {p_value}')

if p_value < 0.05:
    print('There is a significant difference between at least two treatment groups.')
else:
    print('No significant difference detected between treatment groups.')

F-statistic: 3.6959557603686632, p-value: 0.060489765181632134
No significant difference detected between treatment groups.

In [ ]:
f_statistic, p_value = stats.f_oneway(
    df_a['P5v'][df_a['Treatment'] == 'tdp'],
    df_a['P5v'][df_a['Treatment'] == 'tdm'],
    df_a['P5v'][df_a['Treatment'] == 'o']
)

print(f'F-statistic: {f_statistic}, p-value: {p_value}')

if p_value < 0.05:
    print('There is a significant difference between at least two treatment groups.')
else:
    print('No significant difference detected between treatment groups.')


F-statistic: 3.579922376213418, p-value: 0.03273874323572236
There is a significant difference between at least two treatment groups.

In [ ]:
df_v[df_v['Treatment'] == 'cdm']['P6v'].mean()
tv, pv = stats.ttest_ind(df_a[df_a['Treatment'] == 'cdp']['P7a'], 
                         df_a[df_a['Treatment'] == 'tdm']['P7a'])
print(str(pv/2))
print(df_a[df_a['Treatment'] == 'cdm']['P7a'].mean())
print(df_a[df_a['Treatment'] == 'cdp']['P7a'].mean())
print(df_a[df_a['Treatment'] == 'o']['P7a'].mean())
print(df_a[df_a['Treatment'] == 'tdp']['P7a'].mean())
print(df_a[df_a['Treatment'] == 'tdm']['P7a'].mean())

4.861111111111111
0.21964264080582696
4.162162162162162
4.5
4.777777777777778
4.185185185185185
4.791666666666667

In [ ]:
tv, pv = stats.ttest_ind(df_v[df_v['Treatment'] == 'o']['P7v'], 
                         df_v[df_v['Treatment'] == 'tdm']['P7v'])
print(str(pv/2))
print(df_v[df_v['Treatment'] == 'cdm']['P7v'].mean())
print(df_v[df_v['Treatment'] == 'cdp']['P7v'].mean())
print(df_v[df_v['Treatment'] == 'o']['P7v'].mean())
print(df_v[df_v['Treatment'] == 'tdp']['P7v'].mean())
print(df_v[df_v['Treatment'] == 'tdm']['P7v'].mean())

0.13628247215931977
5.416666666666667
6.076923076923077
5.833333333333333
5.64
5.5

In [ ]:
tv, pv = stats.ttest_ind(df_o[df_o['Treatment'] == 'tdm']['P7Q'], 
                         df_o[df_o['Treatment'] == 'cdp']['P7Q'])
print(str(pv/2))
print(df_o[df_o['Treatment'] == 'cdm']['P7Q'].mean())
print(df_o[df_o['Treatment'] == 'cdp']['P7Q'].mean())
print(df_o[df_o['Treatment'] == 'o']['P7Q'].mean())
print(df_o[df_o['Treatment'] == 'tdp']['P7Q'].mean())
print(df_o[df_o['Treatment'] == 'tdm']['P7Q'].mean())

0.0011914426053452653
4.741935483870968
5.303030303030303
5.363636363636363
5.852941176470588
4.096774193548387

In [ ]:
tv, pv = stats.ttest_ind(df_o[df_o['Treatment'] == 'o']['P7A1'], 
                         df_o[df_o['Treatment'] == 'cdp']['P7A1'])
print(str(pv/2))
print(df_o[df_o['Treatment'] == 'cdm']['P7A1'].mean())
print(df_o[df_o['Treatment'] == 'cdp']['P7A1'].mean())
print(df_o[df_o['Treatment'] == 'o']['P7A1'].mean())
print(df_o[df_o['Treatment'] == 'tdp']['P7A1'].mean())
print(df_o[df_o['Treatment'] == 'tdm']['P7A1'].mean())

0.18957271391492847
4.935483870967742
5.909090909090909
5.590909090909091
5.647058823529412
4.838709677419355

In [ ]:
tv, pv = stats.ttest_ind(df_o[df_o['Treatment'] == 'cdp']['P7B1'], 
                         df_o[df_o['Treatment'] == 'tdm']['P7B1'])
print(str(pv/2))
print(df_o[df_o['Treatment'] == 'cdm']['P7B1'].mean())
print(df_o[df_o['Treatment'] == 'cdp']['P7B1'].mean())
print(df_o[df_o['Treatment'] == 'o']['P7B1'].mean())
print(df_o[df_o['Treatment'] == 'tdp']['P7B1'].mean())
print(df_o[df_o['Treatment'] == 'tdm']['P7B1'].mean())

0.02383778287183434
2.903225806451613
3.393939393939394
3.6363636363636362
3.3529411764705883
2.6774193548387095

- For the code and processes involved in the experiment, please refer to the \Files\EX01.ipynb.
- For the f-test to show the correlation of PRH and manipulated pictures, please refer to \Files\f-test.ipynb.
- For the additional ANOVA tests, please refer to \Files\EX02.ipynb.

## Details of Pretests and Main Experiment
Due to space limitations, this section provides additional details about the experiments and preliminary trials that could not be included in the main text of the paper.
### Pretest on Pictures
[Table TM1](\Files\TM1.pdf) (TM1.pdf) illustrates the procedure of the first pretest. The protocol follows Kurdi et al. (2016). In addition to our pretest design, we implemented two attention checks to ensure the quality of responses. The first check was a statement addressing decision-making tendencies. Participants were asked to select a specific response and provide a brief explanation if they had carefully read the preceding questions. Another attention check involved the inclusion of a subtly hidden question prompting participants to select a specific response option, “moderately low,” amongst others.
At the end of the pretest, we incorporated a memory check, prompting participants to choose one from the four pictures which they had not seen. Moreover, we conducted a check of the balance of the personal information among the groups, and the results showed no imbalance.
### Pretest on Text
[Table TM2](\Files\TM2.pdf) (TM2.pdf) illustrate the procedure of the second pretest. The protocol is adopted from Yin et al. (2021). The same attention checks from the first pretest were also applied to the second pretest. The pretest also passed the balance check, similar to the first pretest.
### Main Experiment
[Table TM3](\Files\TM3.pdf) (TM3.pdf) illustrates the procedure of the main experiment, which is similar to the above pretest. We also incorporate the similar attention checks.

## Text-Dic-Report and Img-GPT-Report
In this section, we compare the prediction results of our model with those of GPT-3.5. We calculate the differences between the predicted values and the distances between the prediction distributions as evidence of the practical value of our model’s predictions. The code we used is as follows:

### Text-Dic-Report

In [ ]:
import jieba

# Load the data from both files
df_text_train = pd.read_csv('data/text_train.csv')
df_moessm1_esm = pd.read_csv('data/MOESM1_ESM.csv')

# Extract the list of emotional words from MOESM1_ESM.csv
emotional_words = set(df_moessm1_esm['Word'].tolist())

# Function to calculate the mean valence and arousal for emotional words in a review
def calculate_emotional_means(review, emotional_words, df_moessm1_esm):
    # Tokenize the review into words
    words = jieba.lcut(review)
    # Filter out the emotional words and get their indices in the MOESM1_ESM.csv
    emotional_indices = [df_moessm1_esm.index[df_moessm1_esm['Word'] == word][0] for word in words if word in emotional_words]
    # Calculate mean valence and arousal
    if emotional_indices:
        mean_valence = df_moessm1_esm.loc[emotional_indices, 'Valence_Mean'].mean()
        mean_arousal = df_moessm1_esm.loc[emotional_indices, 'Arousal_Mean'].mean()
    else:
        # If there are no emotional words, return 0/2 as the default values for mean valence and arousal
        mean_valence = 0
        mean_arousal = 2
    return mean_valence, mean_arousal

# Apply the function to each review in the text_train.csv
df_text_train['valence1'], df_text_train['arousal1'] = zip(*df_text_train['review_content'].apply(
    lambda x: calculate_emotional_means(x, emotional_words, df_moessm1_esm)))

# Save the modified dataframe to a new CSV
output_path = 'data/text_train_with_emotional_means.csv'
df_text_train.to_csv(output_path, index=False)

output_path

In [ ]:
# Mapping the values in 'valence1' and 'arousal1' to the range [1, 7] with the specified rules
df_text_train_with_emotional_means_mapped['valence2'] = df_text_train_with_emotional_means_mapped['valence1'].apply(lambda x: round(x + 4))
df_text_train_with_emotional_means_mapped['arousal2'] = df_text_train_with_emotional_means_mapped['arousal1'].apply(lambda x: round((x + 1) * 1.4))

# Save the modified dataframe to a new CSV
output_path_mapped_rules = 'data/text_train_with_emotional_means_mapped_rules.csv'
df_text_train_with_emotional_means_mapped.to_csv(output_path_mapped_rules, index=False)

output_path_mapped_rules

In [ ]:
import numpy as np

# Extract 'valence' and 'valence2' columns from the dataframe
actual_values = df_text_train_with_emotional_means_modified['valence'].values
predicted_values = df_text_train_with_emotional_means_modified['valence2'].values

# Calculate Mean Squared Error (MSE)
mse = np.mean((actual_values - predicted_values) ** 2)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared
r_squared = np.corrcoef(actual_values, predicted_values)[0, 1] ** 2

mse, rmse, r_squared


In [ ]:
# Calculate Mean Absolute Error (MAE)
mae = np.mean(np.abs(actual_values - predicted_values))

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((actual_values - predicted_values) / actual_values) * 100)

mae, mape


In [ ]:
from scipy.stats import ks_2samp

# Calculate the KS-test statistic and p-value for 'valence' and 'valence2' columns
ks_stat, p_value = ks_2samp(df_text_train_with_emotional_means_modified['valence'], df_text_train_with_emotional_means_modified['valence2'])

ks_stat, p_value

from scipy.stats import wilcoxon

# Calculate the Cramér-von Mises distance for 'valence' and 'valence2' columns
cv_distance, cv_p_value = wilcoxon(df_text_train_with_emotional_means_modified['valence'], df_text_train_with_emotional_means_modified['valence2'])

cv_distance, cv_p_value


In [ ]:
from scipy.stats import ks_2samp

# Calculate the KS-test statistic and p-value for 'valence' and 'valence2' columns
ks_stat, p_value = ks_2samp(df_text_train_with_emotional_means_modified['valence'], df_text_train_with_emotional_means_modified['valence2'])

ks_stat, p_value

from scipy.stats import wilcoxon

# Calculate the Cramér-von Mises distance for 'valence' and 'valence2' columns
cv_distance, cv_p_value = wilcoxon(df_text_train_with_emotional_means_modified['valence'], df_text_train_with_emotional_means_modified['valence2'])

cv_distance, cv_p_value


# Valence
(0.8806998071294909, 0.9384560762920612, 0.046242530911141645, 0.704150943408302, 21.220555804250772)

# Arousal
(1.1862474465413284, 1.0891498733146547, 0.0067130068708339844, 0.8799446540815096, 24.117059208348085)

In [ ]:
# Calculate the KS-test statistic and p-value for 'arousal' and 'arousal2' columns
ks_stat_arousal, p_value_arousal = ks_2samp(df_text_train_with_emotional_means_modified['arousal'], df_text_train_with_emotional_means_modified['arousal2'])

# Calculate the Cramér-von Mises distance for 'arousal' and 'arousal2' columns
cv_distance_arousal, cv_p_value_arousal = wilcoxon(df_text_train_with_emotional_means_modified['arousal'], df_text_train_with_emotional_means_modified['arousal2'])

ks_stat_arousal, p_value_arousal, cv_distance_arousal, cv_p_value_arousal

Prompt:
- Please perform Sentiment Classification task. Given the picture, assign a sentiment label from ['Very negative', 'Moderately negative', 'Somewhat negative','Neutral', 'Somewhat positive', 'Moderately positive', 'Very positive']. Return label only without any other text.
  - (We will ask you to rate a series of pictures in terms of how positive or negative they make you feel.)
- Please perform Arousal Classification task. Given the picture, assign a arousal label from ['Very low', 'Moderately low', 'Somewhat low','Neutral', 'Somewhat high', 'Moderately high', 'Very high']. Return label only without any other text.
  - (We will ask you to rate a series of pictures in terms of how excited or aroused they make you feel. In other words, we would like to know how intense the feeling is that picture evokes; whether it is good or bad does not matter.)

In [ ]:
import pandas as pd

# Load the data from the uploaded xlsx file
file_path = 'img-GPT.xlsx'
data = pd.read_excel(file_path)

# Ensure that we only consider the first 30 rows
data = data.head(30)

# Extract the actual values ('pic_valence') and the predicted values ('GPT valence')
actual_values = data['pic_valence']
predicted_values = data['GPT valence']

# Calculate the errors
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean Squared Error (MSE)
mse = mean_squared_error(actual_values, predicted_values)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Coefficient of Determination (R-squared)
r2 = r2_score(actual_values, predicted_values)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(actual_values, predicted_values)

# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((actual_values - predicted_values) / actual_values)) * 100

mse, rmse, r2, mae, mape


In [ ]:
# Correcting the Cramér-von Mises test call and extracting the statistics and p-value properly
cv_result = cramervonmises(predicted_values, 'norm', args=(np.mean(predicted_values), np.std(predicted_values)))

# Extracting the statistics and p-value from the Cramér-von Mises result object
cv_stat = cv_result.statistic
cv_pvalue = cv_result.pvalue

ks_stat, ks_pvalue, cv_stat, cv_pvalue

Again, for additional codes for experimental analysis, f-tests, and ANOVA analysis are included in EX01.ipynb, EX02.ipynb, and f-test.ipynb.

# Quantitative Secondary data
As previously mentioned, this transparency material does not require a detailed report on the data collection process. Due to the sensitive nature of the data and other confidentiality considerations, we are unable to disclose the intermediate datasets used in this study. However, to ensure the reproducibility and rigor of our research, we provide comprehensive documentation of the methods, computations, and code used for data processing and analysis in the subsequent section, *Quantitative Design/Computational Research*. This approach strikes a balance between respecting data privacy and adhering to the principles of transparency in scientific research.

# Quantitative design/computational research
## Decision Tree
 As part of the additional analysis presented in this study, we employed decision tree methods to investigate the relative importance of various aesthetic features of images in influencing the perceived helpfulness of online reviews. This approach allowed us to systematically identify and rank the aesthetic attributes that contribute most significantly to the helpfulness ratings. By leveraging decision trees, we provided a clear, interpretable framework to assess the impact of these visual features, offering valuable insights into the role of image aesthetics in shaping user perceptions and engagement with online reviews.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn as sk

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import seaborn as sns
sns.set()

# evaluation metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# grid search for model selection
from sklearn.model_selection import GridSearchCV

# linear regression
from sklearn.linear_model import LinearRegression

# logistic regression
from sklearn.linear_model import LogisticRegression

# decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import tree

# Import libraries
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import tree

# Load data
data = pd.read_csv('data.csv')
X = data.loc[:,['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']]
y1 = data.loc[:,['PV5']]
y2 = data.loc[:,['PA1']]

# Train a decision tree regression model
regr1 = DecisionTreeRegressor(max_depth=2,min_samples_leaf=0.05, min_samples_split=0.05)
regr1.fit(X, y1)

# Visualize the tree
plt.figure()
tree.plot_tree(regr1, filled=True,feature_names=X.columns,class_names=y1.columns)
plt.show()

regr2 = DecisionTreeRegressor(max_depth=2,min_samples_leaf=0.05, min_samples_split=0.05)
regr2.fit(X, y2)
plt.figure()
tree.plot_tree(regr2, filled=True,feature_names=X.columns,class_names=y2.columns)
plt.show()

## Grabcut
In our image processing pipeline, we employed the GrabCut algorithm to perform segmentation and identify the primary visual subjects within the images. GrabCut is an interactive foreground extraction method that utilizes iterated graph cuts for efficient image segmentation. 

One of the key advantages of GrabCut is its ability to minimize the energy function associated with segmentation, leading to more accurate and efficient extraction of the region of interest. 

By leveraging the capabilities of the GrabCut algorithm, we achieved precise segmentation of images, effectively isolating the visual subjects. This process was instrumental in our analysis, as it allowed for a focused examination of the aesthetic features pertinent to our study.

The implementation of GrabCut in our methodology not only enhanced the accuracy of subject identification but also contributed to the overall efficiency of our image processing workflow. Its iterative approach and ability to preserve edge information ensured that the segmented outputs were of high quality, thereby supporting the robustness of our subsequent analyses.

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('XXX.jpg')
assert img is not None, "file could not be read, check with os.path.exists()"
mask = np.zeros(img.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (0,0,699,524)
cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)

## VGG
In selecting a deep learning model for our study, we utilized the VGG-16 architecture to predict the picture-evoked valence and arousal. VGG-16 is a convolutional neural network renowned for its depth and effectiveness in image classification tasks. It comprises 16 weight layers, including 13 convolutional layers and 3 fully connected layers, enabling it to capture intricate features within images. 

One of the notable advantages of VGG-16 is its high accuracy in image classification, achieving a top-5 test accuracy of 92.7% on the ImageNet dataset, which consists of over 14 million images across nearly 1,000 classes.  This high performance is attributed to its deep architecture, which allows for the learning of complex representations.

Additionally, VGG-16’s architecture is straightforward and well-understood, making it a popular choice for transfer learning applications. Its pre-trained weights can be fine-tuned for specific tasks, enhancing performance even with limited training data. 

In the context of predicting valence and arousal, VGG-16 has been effectively utilized in previous studies. For instance, research has demonstrated its capability in extracting relevant features related to emotional states, facilitating accurate predictions of valence and arousal levels. 

By leveraging VGG-16’s robust feature extraction capabilities and adaptability through transfer learning, our study aimed to accurately model the emotional responses evoked by images, specifically in terms of valence and arousal.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import cv2

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(y_true, K.epsilon(), None)) * 100)

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    img = tf.keras.applications.vgg16.preprocess_input(img)
    return img

def normalize_target(target_values):
    scaler = MinMaxScaler(feature_range=(0, 1))
    target_values = target_values.reshape(-1, 1)
    return scaler.fit_transform(target_values)

def load_data(image_dir, csv_file):
    df = pd.read_csv(csv_file)
    image_paths = df['image_ID'].values
    valence_values = df['valence'].values
    arousal_values = df['arousal'].values
    normalized_valence = normalize_target(valence_values)
    normalized_arousal = normalize_target(arousal_values)
    
    X_data = []
    for image_path in image_paths:
        img_path = os.path.join(image_dir, image_path)
        img = load_and_preprocess_image(img_path)
        X_data.append(img)
    
    X_data = np.array(X_data)
    return X_data, normalized_valence, normalized_arousal

def create_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  
    for layer in base_model.layers[:12]:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    valence_output = Dense(1, name='valence')(x)
    arousal_output = Dense(1, name='arousal')(x)
    model = Model(inputs=base_model.input, outputs=[valence_output, arousal_output])
    model.compile(optimizer=Adam(lr=0.0001), 
                  loss={'valence': rmse, 'arousal': rmse},
                  metrics={'valence': mape, 'arousal': mape})
    return model

def train_model(X_train, y_train_valence, y_train_arousal, X_val, y_valence_val, y_arousal_val, model):
    train = model.fit(X_train, 
                        {'valence': y_train_valence, 'arousal': y_train_arousal},
                        validation_data=(X_val, {'valence': y_valence_val, 'arousal': y_arousal_val}),
                        epochs=10, batch_size=32)
    return train

def main():
    image_dir = 'images'
    csv_file = 'NewDataset01.csv'
    X_data, y_valence, y_arousal = load_data(image_dir, csv_file)
    X_train, X_val, y_train_valence, y_valence_val, y_train_arousal, y_arousal_val = train_test_split(
        X_data, y_valence, y_arousal, test_size=0.2, random_state=5)
    model = create_model()
    history = train_and_save_model(X_train, y_train_valence, y_train_arousal, X_val, y_valence_val, y_arousal_val, model)
    
    val_loss, val_mape_valence, val_mape_arousal = model.evaluate(X_val, {'valence': y_valence_val, 'arousal': y_arousal_val})
    print(f'Validation MAPE (Valence): {val_mape_valence:.4f}%')
    print(f'Validation MAPE (Arousal): {val_mape_arousal:.4f}%')

if __name__ == '__main__':
    main()

# BERT
In this study, we employ the BERT (Bidirectional Encoder Representations from Transformers) model for transfer learning to train a predictive model for text valence and arousal. BERT is a state-of-the-art deep learning model designed for natural language understanding. It utilizes a bidirectional transformer architecture, allowing it to consider the context of words from both the left and the right simultaneously, which is crucial for capturing nuanced meanings in text.

One of the primary advantages of BERT lies in its pre-training on large-scale text corpora, such as the English Wikipedia and BookCorpus, using two unsupervised learning tasks: masked language modeling and next sentence prediction. This pre-training equips the model with a robust understanding of linguistic structures and semantic relationships, making it highly effective for downstream tasks, including sentiment analysis, emotion prediction, and other natural language processing applications.

In the context of our study, BERT’s ability to generate rich, contextualized embeddings for textual data provides a solid foundation for modeling the valence and arousal. By leveraging transfer learning, we fine-tuned the pre-trained BERT model on our specific dataset, enabling it to adapt to the task of predicting emotional states in textual content. This approach not only enhances model performance but also reduces the computational resources and data requirements compared to training a model from scratch.

BERT’s superior performance in various benchmarks, coupled with its flexibility and adaptability, makes it an ideal choice for our research. Its implementation ensures that our analysis is grounded in advanced, reliable, and interpretable methodologies.

In [ ]:
# % 1
from xgboost import XGBClassifier
from bert_serving.client import BertClient
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
tqdm.pandas(desc="Pandas")

# % sklearn
#from sklearn.decomposition import PCA

# load annotated data
a_d = './text_train.xlsx'
df = pd.read_excel(a_d)

# check comment length distribution and choose BERT's max seq length as 30
# df.review_content.apply(lambda x: len(
#     x) if isinstance(x, str)else 0).describe()
'''
count    1325.000000
mean       83.387925
std        79.893124
min        13.000000
25%        24.000000
50%        52.000000
75%       122.000000
max       918.000000
Name: review_content, dtype: float64
'''
df = df[df['review_content'].notna()]
bc = BertClient()
bert_emb = bc.encode(df['review_content'].values.tolist())
bc.close()
bert_emb.shape
with open('./text_train.npy', 'wb') as outfile:
    np.save(outfile, bert_emb)
bert_emb0 = np.load('./text_train.npy')

'''
bert_emb1 = np.insert(bert_emb0, 768, df['fake'], axis=1)
bert_emb2 = np.insert(bert_emb1, 769, df['environment'], axis=1)
bert_emb3 = np.insert(bert_emb2, 770, df['service'], axis=1)
data_a = np.array([df['taste']])
data_b = np.insert(data_a.transpose(),1,df['service'],axis=1)
data_c = np.insert(data_b,2,df['environment'],axis=1)
data_d = np.insert(data_c,1,df['e_score'],axis=1)
'''


def get_clf_report(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    conmat = confusion_matrix(y_test, y_pred)
    conmat = np.mat(conmat)
    print(conmat)
    print(classification_report(y_test, y_pred))
    return y_pred, conmat


X = bert_emb0
Y = df['val4'].astype(str)

# train, test split
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=0, stratify=Y)
y_pred, conmat = get_clf_report(LogisticRegression(
    solver='lbfgs', random_state=0, C=2, class_weight='balanced'))

# SVC


def svm_CV():
    C = [0.1, 1, 10, 100]
    gamma = [1, 0.1, 0.01]
    kernel = ['rbf', 'linear']
    random_grid = {'C': C, 'gamma': gamma, 'kernel': kernel}
    model = SVC(random_state=0, class_weight='balanced')
    svm_random = RandomizedSearchCV(estimator=model,
                                    param_distributions=random_grid,
                                    scoring='f1_weighted',
                                    n_iter=20, cv=5,
                                    verbose=2, random_state=0)
    svm_random.fit(x_train, y_train)

    return svm_random


svm_random = svm_CV()
# get best parameter
parameters = svm_random.best_params_
print(parameters)

clf = SVC(random_state=0, class_weight='balanced', **parameters)
y_pred, conmat = get_clf_report(clf)

# Random Forest


def get_rf_CV():
    n_estimators = [10, 50, 100]
    max_features = ['auto', 'sqrt', 'log2']
    max_depth = [5, 10, 20, 50, None]
    min_samples_split = [8, 16, 32]
    min_samples_leaf = [8, 16, 32]
    bootstrap = [True, False]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    model = RandomForestClassifier(random_state=0, class_weight='balanced')
    rf_random = RandomizedSearchCV(estimator=model,
                                   param_distributions=random_grid,
                                   n_iter=30, cv=5,
                                   scoring='f1_weighted',
                                   verbose=2,
                                   random_state=0)
    rf_random.fit(x_train, y_train)
    return rf_random


rf_random = get_rf_CV()
# get best parameter
parameters = rf_random.best_params_
print(parameters)

clf = RandomForestClassifier(random_state=0,
                             class_weight='balanced',
                             **parameters)
y_pred, conmat = get_clf_report(clf)

# XGBoost


def get_xgb_CV():
    n_estimators = [10, 50, 100, 200, 500]
    max_depth = [2, 5, 10, 20, 50, None]

    random_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth}

    model = XGBClassifier(random_state=0)
    clf_random = RandomizedSearchCV(estimator=model,
                                    param_distributions=random_grid,
                                    n_iter=20, cv=5,
                                    scoring='f1_weighted',
                                    verbose=2,
                                    random_state=0)
    clf_random.fit(x_train, y_train)
    return clf_random


xgb_random = get_xgb_CV()
# get best parameter
parameters = xgb_random.best_params_
print(parameters)

clf = XGBClassifier(random_state=0,
                    **parameters)
y_pred, conmat = get_clf_report(clf)
